In [1]:
import mysql.connector
from mysql.connector.constants import ClientFlag

In [29]:
config = {
    'user': 'root',
    'password': 'Hh12H8y2ppBAq5Cj',
    'host': 'localhost',
    'database': 'data'
}

cnx = mysql.connector.connect(**config)
cnx.autocommit = True
cursor = cnx.cursor()

In [28]:
def markdown_table(res, titles):
    if len(titles) != len(res[0]):
        raise Exception("arg arrays don't match")

    table = ' | '.join(titles)
    table += '\n'
    table += ' | '.join([":---:"] * len(titles))
    table += '\n'

    for row in res:
        table += ' | '.join([str(elem) for elem in row])
        table += '\n'

    return table

# Query 1

In [30]:
cursor.execute("""
SELECT
  q1.AgeGroup as age_group,
  q2.Count * 100.0 / q1.Count AS percentage
FROM
  (
    SELECT
      CASE WHEN party_age BETWEEN 0
      AND 18 THEN 'Underage' WHEN party_age BETWEEN 19
      AND 21 THEN 'Young I' WHEN party_age BETWEEN 22
      AND 24 THEN 'Young II' WHEN party_age BETWEEN 24
      AND 60 THEN 'Adult' WHEN party_age BETWEEN 61
      AND 64 THEN 'Elder I' ELSE 'Elder II' END AS AgeGroup,
      count(1) AS Count
    FROM
      Parties
    GROUP BY
      AgeGroup
  ) q1
  INNER JOIN (
    SELECT
      CASE WHEN p.party_age BETWEEN 0
      AND 18 THEN 'Underage' WHEN p.party_age BETWEEN 18
      AND 21 THEN 'Young I' WHEN p.party_age BETWEEN 21
      AND 24 THEN 'Young II' WHEN p.party_age BETWEEN 24
      AND 60 THEN 'Adult' WHEN p.party_age BETWEEN 60
      AND 64 THEN 'Elder I' ELSE 'Elder II' END AS AgeGroup,
      count(1) AS Count
    FROM
      Parties p
    WHERE
      p.at_fault = true
      AND p.party_type IN (SELECT id FROM PartyType WHERE description='driver')
    GROUP BY
      AgeGroup
  ) q2 ON q1.AgeGroup = q2.AgeGroup""")

In [31]:
res = cursor.fetchall()

In [32]:
for x in res:
    print(x)

('Young I', Decimal('56.28855'))
('Underage', Decimal('56.44656'))
('Adult', Decimal('40.01389'))
('Elder II', Decimal('43.00967'))
('Young II', Decimal('50.83931'))
('Elder I', Decimal('38.93744'))


In [33]:
print(markdown_table(res, ["age_group", "percentage"]))

age_group | percentage
:---: | :---:
Young I | 56.28855
Underage | 56.44656
Adult | 40.01389
Elder II | 43.00967
Young II | 50.83931
Elder I | 38.93744



# Query 4
## Part 1

In [34]:
cursor.execute("""
SELECT
  q1.SeatingPosition AS seating_position,
  q2.Count / q1.Count AS safety_index
FROM
  (
    SELECT
      CASE WHEN victim_seating_position BETWEEN 1
      AND 1 THEN 'Driver' WHEN victim_seating_position BETWEEN 2
      AND 6 THEN 'Passenger' WHEN victim_seating_position BETWEEN 7
      AND 7 THEN 'Station Wagon Rear' WHEN victim_seating_position BETWEEN 8
      AND 8 THEN 'Rear Occupant of Truck or Van' WHEN victim_seating_position BETWEEN 9
      AND 9 THEN 'Position Unknown' WHEN victim_seating_position BETWEEN 0
      AND 0 THEN 'Other Occupants' WHEN victim_seating_position BETWEEN 'A'
      AND 'Z' THEN 'Bus Occupants' ELSE 'Not Stated' END AS SeatingPosition,
      count(1) AS Count
    FROM
      Victims
    GROUP BY
      SeatingPosition
  ) q1
  INNER JOIN (
    SELECT
      CASE WHEN victim_seating_position BETWEEN 1
      AND 1 THEN 'Driver' WHEN victim_seating_position BETWEEN 2
      AND 6 THEN 'Passenger' WHEN victim_seating_position BETWEEN 7
      AND 7 THEN 'Station Wagon Rear' WHEN victim_seating_position BETWEEN 8
      AND 8 THEN 'Rear Occupant of Truck or Van' WHEN victim_seating_position BETWEEN 9
      AND 9 THEN 'Position Unknown' WHEN victim_seating_position BETWEEN 0
      AND 0 THEN 'Other Occupants' WHEN victim_seating_position BETWEEN 'A'
      AND 'Z' THEN 'Bus Occupants' ELSE 'Not Stated' END AS SeatingPosition,
      count(1) AS Count
    FROM
      Victims
    WHERE
      victim_degree_of_injury IN (SELECT id FROM VictimDegreeOfInjury WHERE description='no injury')
    GROUP BY
      SeatingPosition
  ) q2 ON q1.SeatingPosition = q2.SeatingPosition""")

In [35]:
res = cursor.fetchall()

In [ ]:
for x in res:
    print(x)

In [36]:
print(markdown_table(res, ["seating_position", "safety_index"]))

seating_position | safety_index
:---: | :---:
Passenger | 0.7548
Position Unknown | 0.3440
Other Occupants | 0.6244
Rear Occupant of Truck or Van | 0.8177
Station Wagon Rear | 0.8251
Not Stated | 0.1822
Driver | 0.0090



## Part 2

In [37]:
cursor.execute("""
CREATE VIEW SafetyIndexes AS 
    (SELECT
    q1.SeatingPosition,
    q2.Count / q1.Count AS SafetyIndex
    FROM
    (
        SELECT
        CASE WHEN victim_seating_position BETWEEN 1
        AND 1 THEN 'Driver' WHEN victim_seating_position BETWEEN 2
        AND 6 THEN 'Passenger' WHEN victim_seating_position BETWEEN 7
        AND 7 THEN 'Station Wagon Rear' WHEN victim_seating_position BETWEEN 8
        AND 8 THEN 'Rear Occupant of Truck or Van' WHEN victim_seating_position BETWEEN 9
        AND 9 THEN 'Position Unknown' WHEN victim_seating_position BETWEEN 0
        AND 0 THEN 'Other Occupants' WHEN victim_seating_position BETWEEN 'A'
        AND 'Z' THEN 'Bus Occupants' ELSE 'Not Stated' END AS SeatingPosition,
        count(1) AS Count
        FROM
        Victims
        GROUP BY
        SeatingPosition
    ) q1
    INNER JOIN (
        SELECT
        CASE WHEN victim_seating_position BETWEEN 1
        AND 1 THEN 'Driver' WHEN victim_seating_position BETWEEN 2
        AND 6 THEN 'Passenger' WHEN victim_seating_position BETWEEN 7
        AND 7 THEN 'Station Wagon Rear' WHEN victim_seating_position BETWEEN 8
        AND 8 THEN 'Rear Occupant of Truck or Van' WHEN victim_seating_position BETWEEN 9
        AND 9 THEN 'Position Unknown' WHEN victim_seating_position BETWEEN 0
        AND 0 THEN 'Other Occupants' WHEN victim_seating_position BETWEEN 'A'
        AND 'Z' THEN 'Bus Occupants' ELSE 'Not Stated' END AS SeatingPosition,
        count(1) AS Count
        FROM
        Victims
        WHERE
        victim_degree_of_injury IN (SELECT id FROM VictimDegreeOfInjury WHERE description='no injury')
        GROUP BY
        SeatingPosition
    ) q2 ON q1.SeatingPosition = q2.SeatingPosition)""")

ProgrammingError: 1050 (42S01): Table 'SafetyIndexes' already exists

In [38]:
cursor.execute("""
SELECT 
    *
FROM 
    SafetyIndexes
WHERE
    SafetyIndex IN (
          (SELECT MIN(SafetyIndex) FROM SafetyIndexes),
          (SELECT MAX(SafetyIndex) FROM SafetyIndexes)
      )""")

In [39]:
res = cursor.fetchall()

In [30]:
for x in res:
    print(x)

('Station Wagon Rear', Decimal('0.8251'))
('Driver', Decimal('0.0090'))


In [40]:
print(markdown_table(res, ["seating_position", "safety_index"]))

seating_position | safety_index
:---: | :---:
Station Wagon Rear | 0.8251
Driver | 0.0090



# Optimisations

The two commands to get the wanted indexes are:

```python
cursor.execute("CREATE INDEX index_hash_party_type ON Parties(party_type) USING HASH;")
cursor.execute("CREATE INDEX index_hash_victim_injury ON Victims(victim_degree_of_injury) USING HASH;")
```

## Query 1

Before any optimisation, the query:

* Runs in 8.050s before introducting an index
* Does a full scan on the outer relation of the join

Then, after introduction a hash index on the `party_type` of `Parties`:
```python
cursor.execute("CREATE INDEX index_hash_party_type ON Parties(party_type) USING HASH;")
```
We obtain the following results:
* Runs in 1m17s
* Uses the index on the outer relation of the join

We obtain a decrease in performance after optimisation.

## Query 4

The second queries which consists in getting the maximum and minimum values of the table `SafetyIndexes` runs fast and need no optimisation as it is composed only a couple of entries. We will focuse here on optimising the first part of the query: getting the safety indexes for each seating position.

Before any optimisation, the query:

* Runs in 4.590s before introducting an index
* Does a full scan on the outer relation of the join

Then, after introduction a hash index on the `victim_degree_of_injury` of `Victims`:
```python
cursor.execute("CREATE INDEX index_hash_victim_injury ON Victims(victim_degree_of_injury) USING HASH;")
```
We obtain the following results:
* Runs in 33.865s
* Uses the index on the outer relation of the join

We obtain a decrease in performance after optimisation.

In [27]:
cursor.execute("""  
SELECT COUNT(*)
FROM (SELECT statewide_vehicle_type  ,COUNT(Parties.case_id) as collisions ,COUNT(DISTINCT county_city_location) as city
      FROM (Vehicles   INNER JOIN Parties ON Parties.id = Vehicles.party_id
                       INNER JOIN Locations ON Locations.case_id = Parties.id)
GROUP BY statewide_vehicle_type
) A
WHERE A.collisions > 10 AND
      A.city > (
SELECT COUNT(DISTINCT county_city_location)
FROM Locations)/2""")

In [25]:
res = cursor.fetchall()

In [26]:
for x in res:
    print(x)

(1, 'PRIMARY', '<derived2>', None, 'ALL', None, None, None, None, 3452238, 33.33, 'Using where')
(3, 'SUBQUERY', 'Locations', None, 'ALL', None, None, None, None, 3452238, 100.0, None)
(2, 'DERIVED', 'Locations', None, 'ALL', 'case_id', None, None, None, 3452238, 100.0, 'Using temporary; Using filesort')
(2, 'DERIVED', 'Vehicles', None, 'ref', 'statewide_vehicle_type,party_id', 'party_id', '4', 'data.Locations.case_id', 1, 100.0, 'Using index condition')
(2, 'DERIVED', 'Parties', None, 'eq_ref', 'PRIMARY', 'PRIMARY', '4', 'data.Locations.case_id', 1, 100.0, 'Using where')
